**Disclaimer**: This educational content, including any code examples, is provided for instructional purposes only. The author does not endorse or encourage the unauthorised or illegal scraping of websites.

While Python with releveant libraries can be used for web scraping, it's crucial to conduct scraping activities in compliance with applicable laws, the website's terms of service, and ethical considerations. Always review and respect the rules set by the website you are scraping to ensure legal and responsible data collection practices.

# Scraping reviews using Selenium

Here is another example of how Selenium can be used to interact with websites making use of Ajax (Asynchronous JavaScript):

## Selenium is a chrome automation framework

It will enable us to tell chrome:
* go to page bbc.co.uk/weather
* "click the work 'next'"
* scroll down

Selenium will basically open a simplified version of Chrome, for a few seconds, use it and close it afterwards. You might even see it flash on your screen quickly. Then we will use beautiful soup to understand the code.

## BeautifulSoup is an HTML parsing framework

It will enable us to:
* copy the html of the tags eg. div, table
* extract text from these tags

## Getting selenium (don't skip this!)-- You need to download the chromedrive by yourself.

1. find out which version of chrome you have, in chrome open page: chrome://settings/help
2. Go to the list of selenium versions and find folder with your version (eg. 120.0.6099.217). However, the latest version of ChromeDriver is 114.0.5735.90.
**If you are using Chrome version 115 or newer, please consult the [Chrome for Testing availability dashboard](https://googlechromelabs.github.io/chrome-for-testing/). This page provides convenient JSON endpoints for specific ChromeDriver version downloading.**
If your verison is older than 114.0.5735.90, please find you version of ChromeDriver on https://chromedriver.storage.googleapis.com/index.html
3. Go into the folder for your version and download the zip file with the version for your operating system (most likely `chromedriver_mac64.zip` or `chromedriver_win32.zip` ).
4. unzip that file on yoru machine and put it in the folder where this notebook is. unzipped folder and rename it to `chromedriver`. **PLESE DO FORGET TO RENAME IT**.!

In [ ]:
## If you have not installed selenium yet, please uncomment the following line
# !pip install selenium

In [ ]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# define method that will create a browser, suitable to your operating system
import sys
def get_a_browser():
    if sys.platform.startswith('win32') or sys.platform.startswith('cygwin'):
        return webdriver.Chrome() # windows
    else:
        return webdriver.Chrome('./chromedriver') # mac

**Important Note**: allowing your system to run `chromedriver`. This needs to be done just once.

If you are on a mac, you will need to allow your system to use chromium. Run below cell, and you will likely see a warning the first time, click 'cancel' (don't click 'Delete').

After you see the warning, go into `Settings > Security&Privacy > General` and `"Allow Anyway"`.

On a windows pc the process will be simpler. When asked you'll need to allow computer to use the `chrome.exe` file in the folder of `chromedriver` folder.

## Task: let's try to scrape an interactive website

What will be the weather in Edinburgh of next Sunday?

You need a web browser, pen and paper!

In this task you will be asked to do something by yourself (using your web browser, mouse and keyboard), and then you will see how you cen program `Selenium` to do it for you.

**Use www.bbc.co.uk/weather to find out what time will be the sunrise in EDINBURGH next Sunday.**

Do it at least 3 times and observe all the steps you are taking. Make a very detailed list of all the steps, as if you had to describe them to someone over the phone without seeing their screen. See example below.

it will look a bit like this:
* ok, go to www.bbc.co.uk/weather and wait for it to load
* scroll down, do you see a link with words 'Edinburgh' on it? Click it.
* Wait a minute for it to load.
* ok, now scroll down and ...

When you are done with this exercise, we will try to instruct Selenium (Chrome automation tool) to do it for us. Do you think you can try to use Chrome Dev tools to make yoru steps more specific? eg. Instead of saying "copy text in that bold link next to the word Sunrise" try to say "copy text from the html span item with a class `wr-c-astro-data__time`".

**SERIOUSLY: Take a few minutes to do this. It will make you learn more from the below code!**

Ok. And now let's get the python to do it for us.

In [ ]:
browser = get_a_browser()

# the url we want to open
url = u'https://www.bbc.co.uk/weather'

# the browser will start and load the webpage
browser.get(url)

# we wait 1 second to let the page load everything
time.sleep(1)

# we search for an element that is called 'Edinburgh', which is a button
# the button can be clicked with the .click() function
browser.find_element(By.LINK_TEXT,"Edinburgh").click();

# sleep again, let everything load
time.sleep(1)

# we load the HTML body (the main page content without headers, footers, etc.)
body = browser.find_element(By.TAG_NAME,'body')

# we use seleniums' send_keys() function to physically scroll down where we want to click
body.send_keys(Keys.PAGE_DOWN)

# search for the next button to access the next day's weather
try:
    # link will look like "Sun 12Dec" so we use find_element_by_partial_link_text()
    next_button = browser.find_element(By.PARTIAL_LINK_TEXT,'Sun ') 
    next_button.click()
except NoSuchElementException:  #if such element does not exist, just stop looping
    print("something went wrong. There was no Sunday link.")
    
# load current view of the page into a soup
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [ ]:
"""
1. Find all the elements of class pros and print them 
2. These values include today's sunrise and sunset time, and the following 13 days.
3. `browser.page_source` always get the whole page, so we can only find all
4. A not smart, but workable solution is to count how many days between today and next sunday 
   and then choose the right element of all sunrise_tag list.
"""
# The whole list
sunrise_tag = soup.find_all("span", {"class" : 'wr-c-astro-data__time'})
# How many days between today and the next sunday
# PLEASE KEEP THE PAGE OPEN, otherwise the next button will not be found
diff = int(next_button.get_attribute('id')[-1])

print("Sunrise next Sunday: ", sunrise_tag[2*diff].text)

In [ ]:
for i in range(16):
    print(i, sunrise_tag[i].text)

**If you have interests, you can try to find a better way to do this!**